<a href="https://colab.research.google.com/github/Hiroki-ymt/Hiroki-ymt/blob/main/HydroPumpLossCulc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

目的：ポンプ選定のための、配管損失計算を行うプログラム
開発工程案
1. 損失計算機能の作成；Python
2. インターフェイスの作成（条件入力フォームなど）;JavaScript
3. 計算結果出力機能の作成（計算結果表、模式図の自動生成）;Python
4. 報告書の自動生成　；JavaScript？

スプレッドシート読込み

In [4]:
!pip install --upgrade -q gspread

In [5]:
from google.colab import auth
from oauth2client.client import GoogleCredentials
import gspread

auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [10]:
wkb = gc.open('HydroPumpLossCnt')
#シートを読み込む
wks = wkb.worksheet("Controls")
input_data = wks.get_all_records()

print(input_data)

[{'Symbols': 'Q', 'Controls': 10, 'Dimensions': '(L/min)', 'Tytles': '流量'}, {'Symbols': 'D', 'Controls': 250, 'Dimensions': '(mm)', 'Tytles': '配管内径'}, {'Symbols': 'L', 'Controls': 10, 'Dimensions': '(m)', 'Tytles': '配管長'}, {'Symbols': 'C', 'Controls': 140, 'Dimensions': '(-)', 'Tytles': '配管損失係数'}, {'Symbols': 'hmin', 'Controls': 0, 'Dimensions': '(m)', 'Tytles': '最低水位'}, {'Symbols': 'hmax', 'Controls': 15, 'Dimensions': '(m)', 'Tytles': '最高管高さ'}]


計算条件入力

In [29]:
#流量(L/min)
Q = float(wks.acell('B2').value)

#配管内径(mm)
D0 = float(wks.acell('B3').value)
D = D0/1000

#配管長(m)
L = float(wks.acell('B4').value)

#配管損失係数
C = float(wks.acell('B5').value)

#計算上の最低水位と最高配管高さ
hmin = float(wks.acell('B6').value)
hmax = float(wks.acell('B7').value)

print(Q, D, L, C, hmin, hmax)

10.0 0.25 10.0 140.0 0.0 15.0


実揚程

In [30]:
#実揚程
hs = hmax - hmin

print(hs)

15.0


配管摩擦損失　ヘーゼンウイリアムズ式

In [31]:
#ヘーゼンウイリアムズの式
# hp = 10.666 * (Q/60)^1.85 * C^-1.85 * (L/1000)^-4.87

#配管摩擦損失
hp = 10.666 * pow((Q/1000/60),1.85) * pow(C,-1.85) * pow((L/1000),-4.87)

print(hp)

0.6428133684783331


曲がり、レジューサによる損失

In [32]:
#曲がり、レジューサによる損失
he = 5

各種損失水頭の合計演算

In [33]:
#合計損失
h = hs + hp + he

#損失計算の余裕率
eth = 0.1

hc = h*(1 + eth)

print(hc)

22.707094705326167
